In [16]:
import os
import pickle
import pandas as pd
from tqdm import tqdm

from doctr.models import ocr_predictor
from doctr.io import DocumentFile

In [17]:
OCR_DATA_PATH  = './../../data/ocr/docbank/images/'
TXT_DATA_PATH  = './../../results/ocr/linknet_master/'
image_data_dir = os.listdir(OCR_DATA_PATH)

# model = ocr_predictor(det_arch='db_mobilenet_v3_large', reco_arch='crnn_mobilenet_v3_large', pretrained=True)
model = ocr_predictor(det_arch='linknet_resnet50', reco_arch='master', pretrained=True)

In [7]:
import doctr
print(doctr.__version__)

v0.6.0


In [14]:
results = {}

for image_file in tqdm(image_data_dir):
    doc = DocumentFile.from_images(OCR_DATA_PATH + image_file)
    result = model(doc)
    results[image_file] = result
    
with open(TXT_DATA_PATH + 'results.pkl', 'wb') as outp:  # Overwrites any existing file.
    pickle.dump(results, outp, pickle.HIGHEST_PROTOCOL)

100%|████████████████████████████████████████████████████████████████████████| 100/100 [17:25<00:00, 10.46s/it]


In [15]:
with open('../results_mobilenet.pkl', 'rb') as f:
    results = pickle.load(f)

total = {}
for image,result in tqdm(results.items()):
    dim = tuple(reversed(result.pages[0].dimensions))
    predictions = []
    block_id = 0
    for block in result.pages[0].blocks:
        line_id = 0
        for line in block.lines:
            for word in line.words:
                values = []
                geo = word.geometry
                a = list(int(a*b) for a,b in zip(geo[0],dim))
                b = list(int(a*b) for a,b in zip(geo[1],dim))
                values.append(block_id)
                values.append(line_id)
                values.append(word.confidence)
                values.append(a[0])
                values.append(a[1])
                values.append(b[0])
                values.append(b[1])
                values.append(word.value)
                predictions.append(values)
            line_id += 1
        block_id += 1
    total[image] = predictions
    
    
for image,result in tqdm(total.items()):
    name = image[:len(image) - 4]
    df = pd.DataFrame(result, columns = ['block', 'line', 'confidence', 'X1', 'Y1', 'X2', 'Y2', 'token'])
    df.to_csv(TXT_DATA_PATH + name + '.txt', sep=' ',index=False)

100%|███████████████████████████████████████████████████████████████████████| 100/100 [00:00<00:00, 558.33it/s]
